In [15]:
import numpy, matplotlib, time, copy, random
%load_ext line_profiler
directory = '../data/'
file_paths = ['a_example.txt', 'b_read_on.txt', 'c_incunabula.txt','d_tough_choices.txt', 'e_so_many_books.txt', 'f_libraries_of_the_world.txt']

In [2]:
class Library():
    def __init__(self,index,N,T,M, book_ids: set):
        self.id = index
        self.size  = N
        self.signup_time = T
        self.books_per_day = M
        self.book_ids = book_ids
        
    def get_n_best_books(self,n, book_values):
        result = [];
        k=0
        for i in range(len(self.book_ids)):
            if book_values[self.book_ids[i]] != 0:
                result.append(self.book_ids[i])
                k+=1
                if k == n:
                    return tuple(result)
        return tuple(result)
    
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)
        
        

def process_file(filePath):
    with open(directory+filePath,'r') as file:
        content = file.read().split('\n')[:-1];
        B,L,D = content[0].split()
        B,L,D = int(B),int(L),int(D)
    bookValues = [int(n) for n in content[1].split()]#tuple([int(n) for n in content[1].split()])
    libraries = []
    def book_sort(book):
        return bookValues[book]
    for i in range(L):
        N,T,M = content[2+2*i].split()
        N,T,M=int(N),int(T),int(M)
        book_ids = tuple(sorted([int(id) for id in content[2+2*i+1].split()], key = book_sort, reverse=True))
        libraries.append(Library(i,N,T,M,book_ids))
    return ((B,L,D), bookValues, libraries)

In [3]:
def check_solution(D, libraries):
    days = 0
    prev_books = set()
    for library in libraries:
        days+=library.signup_time
        if len(library.book_ids) > (D-days)*library.books_per_day:
            print("what", len(library.book_ids), (D-days)*library.books_per_day)
        assert(not any([(book in prev_books) for book in library.book_ids]))
        prev_books.update(library.book_ids)
    assert(days < D)

In [22]:
def get_best_solution(file_path):
    (B,L,D),book_values, libraries = process_file(file_path)
    
    def library_score(lib):
        delta_time = D - lib.signup_time
        n_best_books = lib.get_n_best_books(delta_time*lib.signup_time, book_values)
        sum_of_best_book_scores = sum([book_values[book] for book in n_best_books])
        return sum_of_best_book_scores/lib.signup_time**(1+lib.signup_time/D)
    
    
    lib_scores = [library_score(lib) for lib in libraries]
    def get_score(selected_lib_ids):
        local_book_values = copy.copy(book_values)
        day = 0
        def book_sort(book):
            return local_book_values[book]
        sorted_libraries = [copy.copy(libraries[i]) for i in selected_lib_ids]
        it=0;
        while it < len(sorted_libraries):
            next_library = sorted_libraries[it]
            day+=next_library.signup_time
            if day>=D:
                break
            next_library.book_ids = next_library.get_n_best_books((D-day)*next_library.books_per_day, local_book_values)
            for book_id in next_library.book_ids:
                local_book_values[book_id] = 0
            it+=1
        score = 0
        for i in range(0,it):
            for idk in sorted_libraries[i].book_ids:
                score+=book_values[idk]    
        check_solution(D, sorted_libraries[:it])
        return score
    
    def get_random_individual():
        indiv = []
        already_chosen = set()
        while len(indiv) < len(libraries):
            ids = random.choices(range(len(libraries)),k = len(libraries), weights = lib_scores)
            for idk in ids:
                if not (idk in already_chosen):
#                     print("###", already_chosen, indiv, idk)
                    already_chosen.add(idk)
                    indiv.append(idk)
        d = 0
        for i in range(len(indiv)):
            d+=libraries[indiv[i]].signup_time
            if d >= D:
                return indiv[:i]
        return indiv
    
    max_score = get_score([i for i in range(len(libraries))])
    for i in range(0,100):
        rand_indiv = get_random_individual()
#         print(rand_indiv)
        score = get_score(rand_indiv)
        if(score > max_score):
            max_score = score
            print(max_score)
    print("Loc max:", max_score)
    return max_score
    
sum_scores = 0
for file_path in file_paths[2:3]:
    print(file_path)
    %lprun -f get_best_solution get_best_solution(file_path)
print("SUM SCORES", sum_scores)

c_incunabula.txt
2151702
2174657
2270352
2316553
Loc max: 2316553
SUM SCORES 0


In [5]:
#IDEAS
return sum_of_best_book_scores/lib.signup_time**(1+lib.signup_time/D)

SyntaxError: 'return' outside function (<ipython-input-5-3155dfd86144>, line 2)